In [0]:
import pandas as pd
flags = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/flags/flag.data',
                    header=None, names = ['name', 'landmass', 'zone', 'area', 
                                                  'population', 'language', 'religion', 
                                                  'bars', 'stripes', 'colors', 'red', 'green',
                  'blue', 'gold', 'white', 'black', 'orange', 'mainhue', 'circles', 'crosses', 
                  'saltires', 'quarters', 'sunstars', 'crescent',
                  'triangle', 'icon', 'animate', 'text', 'topleft', 'botright'])
flags.head()

,name,landmass,zone,area,population,language,religion,bars,stripes,colors,red,green,blue,gold,white,black,orange,mainhue,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,topleft,botright
0,Afghanistan,5,1,648,16,10,2,0,3,5,1,1,0,1,1,1,0,green,0,0,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,1,0,0,1,0,1,0,red,0,0,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,1,1,0,0,1,0,0,green,0,0,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,1,0,1,1,0,0,0,gold,0,0,0,0,0,0,0,0,0,0,blue,red
